## Step 1: Query Crystallographic Open Database

In [ ]:
from __future__ import print_function

from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.calculation import Calculation

from aiida.tools.dbimporters.plugins.cod import CodDbImporter

import ase.io
import numpy as np
import ipywidgets as ipw
from base64 import b64decode
from IPython.display import display, clear_output, Image
from fileupload import FileUploadWidget

import nglview

In [ ]:
atoms = None

layout = ipw.Layout(width="400px")
style = {"description_width":"150px"}

viewer = nglview.NGLWidget()
clear_output()

In [ ]:
structures = [("select structure",{"status":False})]
def on_click_query(b):
    global inp_elements, structures
    importer = CodDbImporter()
    count = 0
    print ("Quering the database ... ", end='')
    for entry in importer.query(formula=str(inp_elements.value)):
        entry_cif = entry.get_cif_node()
        formula = entry_cif.get_ase().get_chemical_formula()
        entry_add = (formula,
                        {
                           "status": True,
                           "cif": entry_cif,
                        }
                    )
        structures.append(entry_add)
        count += 1
    
    print ("%d structures found" % count)
    drop_structure.options = structures
    

In [ ]:
inp_elements = ipw.Text(description="Elements:", value="", placeholder='e.g.: Ni Ti', layout=layout, style=style)
btn_query = ipw.Button(description='Query in CoD')
btn_query.on_click(on_click_query)

display(ipw.HBox([btn_query, inp_elements]))


## Step 2: Select Structure

In [ ]:
def refresh_structure_view():
    global viewer, atoms
    if hasattr(viewer, "component_0"):
        viewer.clear_representations()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)

    viewer.add_component(nglview.ASEStructure(atoms.get_ase())) # adds ball+stick
    viewer.add_unitcell()
    viewer.center()

In [ ]:
def on_change(c):
    global atoms
    indx = c['owner'].index
    new_element = c['new']
    if new_element['status'] is False:
        return
    atoms = new_element['cif']
    formula = atoms.get_ase().get_chemical_formula()
    print (indx, atoms)
    #formula = atoms.get_chemical_formula()
    
    # search for existing structures
    qb = QueryBuilder()
    qb.append(StructureData)
    qb.append(Calculation, filters={'extras.formula':formula}, descendant_of=StructureData)
    qb.order_by({Calculation:{'ctime':'desc'}})
    for n in qb.iterall():
        calc = n[0]
        print("Found existing calculation: PK=%d | %s"%(calc.pk, calc.get_extra("structure_description")))
        thumbnail = b64decode(calc.get_extra("thumbnail"))
        display(Image(data=thumbnail))

    refresh_structure_view()


drop_structure = ipw.Dropdown(description="Select the Structure:", options=structures, style=style, layout=layout )
drop_structure.observe(on_change, names='value')
display(drop_structure, ipw.VBox([viewer]))

## Step 3: Store in AiiDA Database

In [ ]:
def on_click_store(b):
    global atoms
    with store_out:
        clear_output()
        if atoms is None:
            print ("Specify a structure first!")
            return
        #AiiDA requires structures to have cell
#       if np.all(atoms.cell == 0.0):
#           atoms.center(vacuum=0.1)
        if data_format.value is 'CifData':
            s=atoms.copy()
        elif data_format.value is 'StructureData':
            s = StructureData(ase=atoms.get_ase())
            # ensure that tags got correctly translated into kinds 
            for t1, k in zip(atoms.get_ase().get_tags(), s.get_site_kindnames()):
                t2 = int(k[-1]) if k[-1].isnumeric() else 0
                assert t1==t2
            s.description = inp_descr.value
        
        s.store()
        print("Stored in AiiDA: "+repr(s))

inp_descr = ipw.Text(placeholder="Description (optional)")   
btn_store = ipw.Button(description='Store in AiiDA')
btn_store.on_click(on_click_store)
data_format = ipw.RadioButtons(
    options=['CifData', 'StructureData'],
#     value='pineapple',
    description='Data type:',
    disabled=False
)


store_out = ipw.Output()
display(data_format, ipw.HBox([btn_store, inp_descr]), store_out)